In [2]:
import pandas as pd

In [3]:

# 讀取上傳的 Excel 檔案
NST_crawler_file_path = './NST_crawler.xlsx'
NST_crawler_df = pd.read_excel(NST_crawler_file_path)

# 顯示 DataFrame 的前幾行，以確認讀取正確
NST_crawler_df.head()


,計畫主持人,學校,碩士畢業學年度,碩士畢業學校,碩士指導教授,碩士論文題目,博士畢業學年度,博士畢業學校,博士指導教授,博士論文題目,...,查獲人數,查獲博士人數,博士畢業學年度_2,博士畢業學校_2,博士指導教授_2,博士論文題目_2,博士畢業學年度_3,博士畢業學校_3,博士指導教授_3,博士論文題目_3
0,Brij Bhooshan Gupta,亞洲大學資訊工程學系,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Hieu,國立臺北科技大學電子工程系,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,98,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Naveed Ishtiaq Chaudhary,國立雲林科技大學前瞻學士學位學程,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,丁川康,國立清華大學動力機械工程學系,84.0,國立清華大學／動力機械學系,林士傑,以尺寸文字為基礎進行機械製圖尺寸線辨識之研究,NaN,NaN,NaN,NaN,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,丁信文,國立高雄科技大學電機與資訊學院電子工程系(建工校區),92.0,國立成功大學／電機工程學系碩博士班,劉濱達、張順志,應用於類比數位轉換器SNDR及ENOB測試之時域內建自我測試方法,NaN,NaN,NaN,NaN,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 處理資料

## 處理成 RDF 格式
學生姓名 - 博士指導教授 - 學校 - 畢業年度 - 論文名稱


In [27]:
import pandas as pd
import numpy as np

# 假設 NST_crawler_df 是您的原始DataFrame

RDF_df = pd.DataFrame()

for index, row in NST_crawler_df.iterrows():
    
    # 檢查碩士資料是否不為空或NaN
    if pd.notnull(row["碩士畢業學年度"]) or pd.notnull(row["碩士畢業學校"]) or pd.notnull(row["碩士論文題目"]):
        current_dict = {
            "學生姓名": row["計畫主持人"],
            "畢業學年度": int(row[f"碩士畢業學年度"]),
            "畢業學校": row["碩士畢業學校"],
            "論文題目": row["碩士論文題目"],
            "學籍": "碩士",
            "計畫發表學校": row["學校"]
        }
        RDF_df = pd.concat([RDF_df, pd.DataFrame([current_dict])], ignore_index=True)
    
    # 處理博士資料
    for PHD_count in range(int(row["查獲博士人數"])):
        
        endWith = f"_{PHD_count}" if PHD_count > 1 else ""
        
        current_dict = {
            "學生姓名": row["計畫主持人"],
            "畢業學年度": int(row[f"博士畢業學年度{endWith}"]),
            "畢業學校": row[f"博士畢業學校{endWith}"],
            "論文題目": row[f"博士論文題目{endWith}"],
            "學籍": "博士",
            "計畫發表學校": row["學校"]
        }
        RDF_df = pd.concat([RDF_df, pd.DataFrame([current_dict])], ignore_index=True)

# 處理重複
columns_to_check = ["學生姓名", "畢業學年度", "畢業學校", "論文題目", "學籍"]
RDF_df = RDF_df.drop_duplicates(subset=columns_to_check, keep="first")

# 保存至Excel
RDF_df.to_excel('NST_crawler_RDF.xlsx', index=False, engine='openpyxl', sheet_name="研究人才")
RDF_df


,學生姓名,畢業學年度,畢業學校,論文題目,學籍,計畫發表學校
0,丁川康,84,國立清華大學／動力機械學系,以尺寸文字為基礎進行機械製圖尺寸線辨識之研究,碩士,國立清華大學動力機械工程學系
1,丁信文,92,國立成功大學／電機工程學系碩博士班,應用於類比數位轉換器SNDR及ENOB測試之時域內建自我測試方法,碩士,國立高雄科技大學電機與資訊學院電子工程系(建工校區)
2,丁英智,97,國立交通大學／資訊科學與工程研究所,模糊邏輯控制於語者調適及音訊事件偵測之參數調適,博士,國立虎尾科技大學電機工程系
3,丁國正,106,逢甲大學／資訊工程學系,基於類神經網路與天際線查詢技術之 工業4.0 實踐方法,博士,國立雲林科技大學前瞻學士學位學程
4,方士豪,101,國立成功大學／電機工程學系碩博士班,利用子空間演算法之區塊傳送系統盲蔽式通道估測研究,博士,元智大學電機工程學系（所）
...,...,...,...,...,...,...
1259,欉振坤,102,國立中正大學／資訊工程研究所,改善關鍵字拍賣中廣義二級價格的穩健度,博士,國立勤益科技大學資訊工程系
1260,龔于農,95,長庚大學／資訊工程研究所,應用隱藏式馬可夫模型於新興流行性感冒病毒之分類,碩士,長庚大學新興病毒感染研究中心
1261,龔于農,101,長庚大學／電機工程學系,識別A型流感病毒之共同演化氨基酸位點,博士,長庚大學新興病毒感染研究中心
1262,龔旭陽,78,國立清華大學／資訊科學研究所,使用分散式方法設計一階層式移動性無線電網路,碩士,國立屏東科技大學資訊管理系
